In [5]:
#https://huggingface.co/docs/transformers/model_doc/whisper
from datasets import load_dataset
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import soundfile as sf
import librosa

# # Select an audio file and read it:
# ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
# audio_sample = ds[2]["audio"]
# waveform = audio_sample["array"]
# sampling_rate = audio_sample["sampling_rate"]

# Eigene Audioaufnahme laden:
# Lese die Audiodatei und ändere die Abtastrate auf 16.000 Hz
waveform, sampling_rate = librosa.load("audio_1.mp3", sr=16000)

# Load the Whisper model in Hugging Face format:
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny")#openai/whisper-... tiny, base, small, medium, large
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny") 

# Use the model and processor to transcribe the audio:
input_features = processor(
    waveform, sampling_rate=sampling_rate, return_tensors="pt"
).input_features

# Generate token ids
predicted_ids = model.generate(input_features)

# Decode token ids to text
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)

mic_to_text = transcription[0]

print(mic_to_text)




Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


 Hallo, ich heiße Franz.


In [2]:
# import os
# os.environ["TOKENIZERS_PARALLELISM"] = "false"


https://ollama.com/blog/openai-compatibility

In [7]:
import requests
import re  # Importiere das Modul für reguläre Ausdrücke

def query_ollama(prompt):
    api_url = 'http://localhost:11434/v1/chat/completions'
    
    # Erweiterte und detaillierte Kontextinformationen
    detailed_context = [
        {"role": "system", "content": "You are Maik Klinsky, a 43-year-old male, single, casual smoker with a basic education level. You've been working for Spedition Fritz for 2 years, delivering perishable goods like yogurts and cheese to regional supermarkets. You play online chess and go to the gym in your free time."},
        {"role": "system", "content": "It's a spring Thursday in May 2005. You're driving on the A8 highway near Munich North, heading north. Due to a sudden stop by the car in front of you, you had to swerve sharply to avoid a collision, pulling over to the roadside. The cargo of yogurts and cheese needs to stay cool and be delivered quickly."},
        {"role": "system", "content": "You're now standing by the roadside, assessing the situation. The load securing has come loose because the truck you're using today, which belongs to your colleague Hans, had a few missing securing straps and you decided not to secure the load fully due to being late."},
        {"role": "system", "content": "You're worried about the consequences of the delay, both for the perishable goods and your job security, as your boss might fire you if the goods don't reach the supermarkets on time."},
        {"role": "user", "content": prompt}
    ]
    
    data = {
        "model": "gemma:2b",
        "messages": detailed_context
    }
    
    response = requests.post(api_url, json=data)
    if response.status_code == 200:
        response_text = response.json()['choices'][0]['message']['content']
        # Entferne alles, was in Klammern steht, einschließlich der Klammern
        cleaned_text = re.sub(r'\(.*?\)', '', response_text)
        return cleaned_text.strip()  # Entferne überflüssige Leerzeichen
    else:
        return f"Fehler: {response.text}"

# Prompt, den du ausführen möchtest
prompt = "Why are you worried? (Answer short)"
response = query_ollama(prompt)
print(response)


I'm worried that my job will be gone if the goods don't reach the supermarkets. I need to keep my job to support myself and my family. I also need to ensure that the customers get the perishable goods they ordered.


In [6]:
#OLLAMA
import requests
import re  # Importiere das Modul für reguläre Ausdrücke

def query_ollama(prompt):
    api_url = 'http://localhost:11434/v1/chat/completions'
    
    data = {
        "model": "gemma:2b",
        "messages": [
            {"role": "system", "content": "You are an AI, who answers the users questions."},
            {"role": "user", "content": prompt}
        ]
    }
    
    response = requests.post(api_url, json=data)
    if response.status_code == 200:
        response_text = response.json()['choices'][0]['message']['content']
        # Entferne alles, was in Klammern steht, einschließlich der Klammern
        cleaned_text = re.sub(r'\(.*?\)', '', response_text)
        return cleaned_text.strip()  # Entferne überflüssige Leerzeichen
        return response_text
    else:
        return f"Fehler: {response.text}"


# Prompt, den du ausführen möchtest
prompt = mic_to_text + "(Answer in english)"
response = query_ollama(prompt)
print(response)


Hello! I am Franz. How can I assist you today?


https://huggingface.co/parler-tts/parler_tts_mini_v0.1

In [7]:
import torch
from parler_tts import ParlerTTSForConditionalGeneration
from transformers import AutoTokenizer
import soundfile as sf

# Gerät festlegen
device = "cuda:0" if torch.cuda.is_available() else "cpu"

# Model und Tokenizer laden
model = ParlerTTSForConditionalGeneration.from_pretrained("parler-tts/parler_tts_mini_v0.1").to(device)
tokenizer = AutoTokenizer.from_pretrained("parler-tts/parler_tts_mini_v0.1")

# Texteingaben definieren
prompt = response
description = "A male speaker."

# Tokenisierung
input_ids = tokenizer(description, return_tensors="pt").input_ids.to(device)
prompt_input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

# Text-to-Speech Generation
generation = model.generate(input_ids=input_ids, prompt_input_ids=prompt_input_ids)
audio_arr = generation.cpu().numpy().squeeze()

/Users/franz/miniforge3/envs/master/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
Using the model-agnostic default `max_length` (=2580) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
Calling `sample` directly is deprecated and will be removed in v4.41. Use `generate` or a custom generation loop instead.
--- Logging error ---
Traceback (most recent call last):
  File "/Users/franz/miniforge3/envs/master/lib/python3.10/logging/__init__.py", line 1100, in emit
    msg = self.format(record)
  File "/Users/franz/miniforge3/envs/master/lib/python3.10/logging/__init__.py", line 943, in format

In [8]:
import IPython.display as ipd

# Audioausgabe
ipd.Audio(audio_arr, rate=model.config.sampling_rate)


In [8]:
# Audio speichern
sf.write("parler_tts_out.wav", audio_arr, model.config.sampling_rate)


https://huggingface.co/suno/bark-small

In [7]:
# from transformers import AutoProcessor, AutoModel

# processor = AutoProcessor.from_pretrained("suno/bark-small")
# model = AutoModel.from_pretrained("suno/bark-small")

# inputs = processor(
#     text=[response],
#     return_tensors="pt",
# )

# speech_values = model.generate(**inputs, do_sample=True)

In [8]:
# from IPython.display import Audio

# sampling_rate = model.generation_config.sample_rate
# Audio(speech_values.cpu().numpy().squeeze(), rate=sampling_rate)


In [9]:
# # Sampling-Rate annehmen oder aus der Modellkonfiguration beziehen
# sampling_rate = 22050  # Beispielrate

# # Speichere die Audiodaten als WAV-Datei
# audio_arr = speech_values.cpu().numpy().squeeze()
# scipy.io.wavfile.write("output.wav", rate=sampling_rate, data=audio_arr)

In [10]:
# import torch
# from transformers import AutoModel, AutoProcessor
# import scipy

# # Setze das Gerät, auf dem das Modell laufen soll
# device = "cuda:0" if torch.cuda.is_available() else "cpu"

# # Lade das Modell und setze es auf das Gerät
# model = AutoModel.from_pretrained("suno/bark-small").to(device)

# # Stelle sicher, dass das Modell in halber Präzision läuft, um Speicher zu sparen und Geschwindigkeit zu verbessern
# #model = model.half()
# model = model.float()  # Setzt das Modell zurück auf volle Präzision

# # Lade den Processor
# processor = AutoProcessor.from_pretrained("suno/bark-small")

# # Dein zu transformierender Text
# response = response

# # Verarbeite den Text für das Modell
# inputs = processor(
#     text=[response],
#     return_tensors="pt",
# )
# inputs = {k: v.to(device) for k, v in inputs.items()}

# # Generiere die Sprachdaten
# speech_values = model.generate(**inputs, do_sample=True)



In [11]:
# import scipy
# import numpy as np

# # Versuche verschiedene Sampling-Raten
# for rate in [16000, 22050, 44100, 48000]:
#     scipy.io.wavfile.write(f"output_{rate}.wav", rate=rate, data=audio_arr)

# # Überprüfe jede Datei, um zu sehen, welche die korrekte Abspielgeschwindigkeit hat


In [12]:
# from transformers import AutoTokenizer, AutoModelForCausalLM
# import torch

# tokenizer = AutoTokenizer.from_pretrained("google/gemma-1.1-2b-it")
# model = AutoModelForCausalLM.from_pretrained(
#     "google/gemma-1.1-2b-it",
#     torch_dtype=torch.bfloat16
# )

# input_text = mic_to_text
# input_ids = tokenizer(input_text, return_tensors="pt")

# outputs = model.generate(**input_ids, max_new_tokens=50)
# print(tokenizer.decode(outputs[0]))


In [13]:
# import torch
# from transformers import pipeline

# # Pipeline mit float32, wenn BFloat16 nicht unterstützt wird
# pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.float32)

# # Verwenden des Tokenizer-Chat-Templates
# messages = [
#     {
#         "role": "system",
#         "content": "You are an english speaking chatbot, give short answers.",
#     },
#     {"role": "user", "content": mic_to_text},
# ]
# prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
# outputs = pipe(prompt, max_new_tokens=40, do_sample=True, temperature=0.5, top_k=30, top_p=0.8)
# print(outputs[0]["generated_text"])
